In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, floor, log10


def round_sig(x: float, sig: int) -> float:
    """Rounds x to sig significant digits"""
    return x if x == 0 else round(x, sig - int(floor(log10(abs(x)))) - 1)


plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (20, 28)
# plt.style.use('dark_background')

### Ładowanie danych

In [ ]:
df = pd.read_csv("data.csv")
df.head()

### Obliczenie różnicy temperatur w °C i kelwinach (to to samo)

In [ ]:
df["T_k"] = df.loc[:, "Tg"] - df.loc[:, "Td"]
print(df)

### Obliczenie niepewności $u(T_g)$ i $u(T_d)$

In [ ]:
# Napięcie stałe DC
# u(T) = += (0.5% * val + 0.001) / sqrt(3)

Tg = list(df.iloc[:, 2])
Td = list(df.iloc[:, 1])

u_Tg = [round_sig((0.005 * x + 0.001) / sqrt(3), 2) for x in Tg]
u_Td = [round_sig((0.005 * x + 0.001) / sqrt(3), 2) for x in Td]

print(u_Tg)

### Obliczenie niepewności $u(\Delta T)$ z prawa przenoszenia niepewności

In [ ]:
# T = Tg - Td
# sqrt(u(Tg)^2 + u(Td)^2)

u_deltaT = [round_sig(sqrt(u_tg**2 + u_td**2), 2) for u_tg, u_td in zip(u_Tg, u_Td)]
print(u_deltaT)

### Obliczenie niepewności natężenia $I$

In [ ]:
# MY-67 natężenie
# zakres = 10A, rozdzielczość = 0.01 A
# u(I) = +- 2% * val + 5 * roz
# u(I) = +- 1/50 * val + 0.05

u_I = [round_sig(1 / 50 * i + 0.05, 2) for i in df["I"]]
print(u_I)

### Wykres $f(I) = (\Delta T)$ i regresja liniowa

In [ ]:
plt.errorbar(
    df["I"],
    df["T_k"],
    xerr=u_I,
    yerr=u_deltaT,
    fmt="o",
    label="xd",
    linewidth=1,
    markersize=5,
    color="red",
)

a, b = np.polyfit(df["I"], df["T_k"], 1)
a = round_sig(a, 3)
b = round_sig(b, 3)
plt.plot(df["I"], a * df["I"] + b, color="orange")


def get_a_error(X: list[float], Y: list[float]) -> float:
    """Zwraca niepewność współczynnika kierukowego a"""

    n = len(X)
    S_x = sum(X)
    S_xx = sum([x * x for x in X])
    S_ee = sum([(y - a * x - b) ** 2 for x, y in zip(X, Y)])
    return sqrt(n / (n - 2) * S_ee / (n * S_xx - S_x * S_x))


def get_b_error(X: list[float], Y: list[float]) -> float:
    """Zwraca niepewność wyrazu wolnego b"""

    n = len(X)
    S_x = sum(X)
    S_xx = sum([x * x for x in X])
    S_ee = sum([(y - a * x - b) ** 2 for x, y in zip(X, Y)])
    return sqrt(1 / (n - 2) * (S_xx * S_ee) / (n * S_xx - S_x * S_x))


a_error = round_sig(get_a_error(df["I"], df["T_k"]), 2)
b_error = round_sig(get_b_error(df["I"], df["T_k"]), 2)

print(f"a={a}, u(a)={a_error}")
print(f"b={b}, u(b)={b_error}")


plt.title(
    r"Wykres zależności różnicy temperatur $\Delta T$ od napięcia $I$",
    fontsize=30,
)
plt.ylabel("r$Delta T, K$", fontsize=30)
plt.xlabel("$I$, A", fontsize=30)

plt.savefig("./img/wykres.jpg")

### Obliczenie róznicy temperatur $\Delta T$ dla $I$ = 1A

In [ ]:
# delta_T = a * I + b
# dla I = 1

res = round_sig(a * 1 + b, 3)
print(f"Różnica temperatur na module Peltiera dla I = 1A wynosi {res}")